# Notebook prep from Chap 9 Train, Validate, Test

In [ ]:
import pandas as pd
import numpy as np
from typing import Sequence

from sklearn.ensemble import RandomForestRegressor
from pandas.api.types import is_string_dtype, is_object_dtype, is_categorical_dtype
from sklearn.metrics import mean_absolute_error, mean_squared_log_error, mean_squared_error, r2_score

from rfpimp import *  # feature importance plot

bookcolors = {
    'crimson': '#a50026', 'red': '#d73027',
    'redorange': '#f46d43', 'orange': '#fdae61',
    'yellow': '#fee090', 'sky': '#e0f3f8',
    'babyblue': '#abd9e9', 'lightblue': '#74add1',
    'blue': '#4575b4', 'purple': '#313695'}

def test(X, y, n_estimators=50,
         max_features='auto', min_samples_leaf=1):
    rf = RandomForestRegressor(n_estimators=n_estimators,
                               n_jobs=-1,
                               oob_score=True,
                               max_features=max_features, 
                               min_samples_leaf=min_samples_leaf)
    rf.fit(X, y)
    oob = rf.oob_score_
    n = rfnnodes(rf)
    h = np.median(rfmaxdepths(rf))
    print(f"OOB R^2 {oob:.5f} using {n:,d} tree nodes with {h} median tree height")
    return rf, oob
        
def df_split_dates(df,colname):
    df["saleyear"] = df[colname].dt.year
    df["salemonth"] = df[colname].dt.month
    df["saleday"] = df[colname].dt.day
    df["saledayofweek"] = df[colname].dt.dayofweek
    df["saledayofyear"] = df[colname].dt.dayofyear
    df[colname] = df[colname].astype(np.int64) # convert to seconds since 1970
    # age can be nan since YearMade can be nan
    df['age'] = df['saleyear'] - df['YearMade'] # synthesize age

def extract_sizes(df, colname):
    df[colname] = df[colname].str.extract(r'([0-9.]*)', expand=True)
    df[colname] = df[colname].replace('', np.nan)
    df[colname] = pd.to_numeric(df[colname])
    
def df_normalize_strings(df):
    for col in df.columns:
        if is_string_dtype(df[col]) or is_object_dtype(df[col]):
            df[col] = df[col].str.lower()
            df[col] = df[col].fillna(np.nan) # make None -> np.nan
            df[col] = df[col].replace('none or unspecified', np.nan)
            df[col] = df[col].replace('none', np.nan)
            df[col] = df[col].replace('#name?', np.nan)
            df[col] = df[col].replace('', np.nan)

def df_cat_to_catcode(df:pd.DataFrame):
    for colname in df.columns:
        if is_categorical_dtype(df[colname]):
            df[colname] = df[colname].cat.codes + 1
            
def fix_missing_num(df, colname):
    df[colname+'_na'] = pd.isnull(df[colname])
    df[colname].fillna(df[colname].median(), inplace=True)

In [ ]:
def clean(df):
    del df['MachineID'] # dataset has inconsistencies
    del df['SalesID']   # unique sales ID so not generalizer

    df['auctioneerID'] = df['auctioneerID'].astype(str)

    df_normalize_strings(df)

    extract_sizes(df, 'Tire_Size')
    extract_sizes(df, 'Undercarriage_Pad_Width')

    df.loc[df['YearMade']<1950, 'YearMade'] = np.nan
    df.loc[df.eval("saledate.dt.year < YearMade"), 'YearMade'] =         df['saledate'].dt.year

    df.loc[df.eval("MachineHoursCurrentMeter==0"),
           'MachineHoursCurrentMeter'] = np.nan

In [ ]:
def df_order_product_size(df):
    sizes = {np.nan:0, 'mini':1, 'compact':1, 'small':2, 'medium':3,
             'large / medium':4, 'large':5}
    df['ProductSize'] = df['ProductSize'].map(sizes).values

In [ ]:
def onehot(df, colname):
    ascat = df[colname].astype('category').cat.as_ordered()
    onehot = pd.get_dummies(df[colname], prefix=colname, dtype=bool)
    del df[colname]
    df = pd.concat([df, onehot], axis=1)
    # return altered dataframe and column training categories
    return df, ascat.cat.categories

In [ ]:
def split_fiProductClassDesc(df):
    df_split = df.fiProductClassDesc.str.split(' - ',expand=True).values
    df['fiProductClassDesc'] = df_split[:,0] 
    df['fiProductClassSpec'] = df_split[:,1] # temporary column
    pattern = r'([0-9.\+]*)(?: to ([0-9.\+]*)|\+) ([a-zA-Z ]*)'
    spec = df['fiProductClassSpec']
    df_split = spec.str.extract(pattern, expand=True).values
    df['fiProductClassSpec_lower'] = pd.to_numeric(df_split[:,0])
    df['fiProductClassSpec_upper'] = pd.to_numeric(df_split[:,1])
    df['fiProductClassSpec_units'] = df_split[:,2]
    del df['fiProductClassSpec'] # remove temporary column

In [ ]:
def feature_eng(X): # for later use
    df_split_dates(X, 'saledate')
    df_order_product_size(X)
    split_fiProductClassDesc(X)

    X, hf_cats = onehot(X, 'Hydraulics_Flow')
    # normalize categories first then one-hot encode
    X['Enclosure'] = X['Enclosure'].replace('erops w ac', 'erops ac')
    X['Enclosure'] = X['Enclosure'].replace('no rops', np.nan)
    X, enc_cats = onehot(X, 'Enclosure')
    catencoders = {'Hydraulics_Flow':hf_cats,
                   'Enclosure':enc_cats}

    return X, catencoders

In [ ]:
def df_fix_missing_nums(df:pd.DataFrame) -> dict:
    medians = {}  # column name to median
    for colname in df.columns:
        if is_numeric_dtype(df[colname]):
            medians[colname] = df[colname].median(skipna=True)
            fix_missing_num(df, colname)
    return medians

In [ ]:
def df_string_to_cat(df:pd.DataFrame) -> dict:
    catencoders = {}
    for colname in df.columns:
        if is_string_dtype(df[colname]) or is_object_dtype(df[colname]):
            df[colname] = df[colname].astype('category').cat.as_ordered()
            catencoders[colname] = df[colname].cat.categories
    return catencoders

In [ ]:
def numericalize(X, catencoders):
    medians = df_fix_missing_nums(X)            
    e = df_string_to_cat(X)
    catencoders.update(e)
    df_cat_to_catcode(X)    
    return medians

In [ ]:
df = pd.read_feather("data/bulldozer-train.feather")
df = df.iloc[-100_000:] # same 100,000 records as before
X, y = df.drop('SalePrice', axis=1), df['SalePrice']

In [ ]:
y = np.log(y)
clean(X)
X, catencoders = feature_eng(X)
medians = numericalize(X, catencoders)

In [ ]:
rf, r2_train = test(X, y, n_estimators=150)

In [ ]:
def df_fix_missing_test_nums(df_test, medians):
    for colname in medians:
        df_test[colname+'_na'] = pd.isnull(df_test[colname])
        df_test[colname].fillna(medians[colname], inplace=True)

In [ ]:
def df_apply_cats(df_test:pd.DataFrame, catencoders:dict):
    for colname,encoder in catencoders.items():
        # encode with categories from training set
        df_test[colname] =             pd.Categorical(df_test[colname],
                           categories=encoder, ordered=True)

In [ ]:
def onehot_apply_cats(df_test, colname, catencoders):
    df_test[colname] =         pd.Categorical(df_test[colname],
                       categories=catencoders[colname],
                       ordered=True)
    onehot = pd.get_dummies(df_test[colname], prefix=colname, dtype=bool)
    del df_test[colname]
    df_test = pd.concat([df_test, onehot], axis=1)
    del catencoders[colname] # simplify df_apply_cats()
    return df_test

In [ ]:
def feature_eng_test(df_test, catencoders):
    df_split_dates(df_test, 'saledate')
    df_order_product_size(df_test)
    split_fiProductClassDesc(df_test)

    df_test = onehot_apply_cats(df_test, 'Hydraulics_Flow', catencoders)
    df_test['Enclosure'] = df_test['Enclosure'].replace('erops w ac', 'erops ac')
    df_test['Enclosure'] = df_test['Enclosure'].replace('no rops', np.nan)
    df_test = onehot_apply_cats(df_test, 'Enclosure', catencoders)

    return df_test

In [ ]:
def numericalize_test(df_test:pd.DataFrame, medians:dict, catencoders:dict):
    df_apply_cats(df_test, catencoders)
    df_fix_missing_test_nums(df_test, medians)
    df_cat_to_catcode(df_test)

In [ ]:
df_valid = pd.read_feather("data/bulldozer-valid.feather")
X_valid, y_valid = df_valid.drop('SalePrice', axis=1), df_valid['SalePrice']

y_valid = np.log(y_valid)
clean(X_valid)
X_valid = feature_eng_test(X_valid, catencoders)
numericalize_test(X_valid, medians, catencoders)

In [ ]:
X_valid = X_valid.reindex(columns=X.columns)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

y_pred = rf.predict(X_valid)
# Use np.exp(y_valid) to get back into dollars space
mae_valid_baseline = mean_absolute_error(np.exp(y_valid), np.exp(y_pred))
rmsle_valid_baseline = np.sqrt( mean_squared_error(y_valid, y_pred) )
r2_valid_baseline = rf.score(X_valid, y_valid)
print(f"Validation R^2 {r2_valid_baseline:.5f}, "+
      f"RMSLE {rmsle_valid_baseline:.5f}, "+
      f"MAE ${mae_valid_baseline:.0f}")

In [ ]:
df = pd.read_feather("data/bulldozer-train.feather")
df = df.query('saledate.dt.year>=2007').copy()
X, y = df.drop('SalePrice', axis=1), df['SalePrice']

In [ ]:
y = np.log(y)
clean(X)
X, catencoders = feature_eng(X)
medians = numericalize(X, catencoders)

df_valid = pd.read_feather("data/bulldozer-valid.feather")
X_valid, y_valid = df_valid.drop('SalePrice', axis=1), df_valid['SalePrice']
y_valid = np.log(y_valid)
clean(X_valid)
X_valid = feature_eng_test(X_valid, catencoders)
df_apply_cats(X_valid, catencoders)
df_fix_missing_test_nums(X_valid, medians)
df_cat_to_catcode(X_valid)

In [ ]:
def test_valid(X, y, X_valid, y_valid, n_estimators=200,
               max_features='auto', min_samples_leaf=1):
    X_valid = X_valid.reindex(columns=X.columns)
    rf = RandomForestRegressor(n_estimators=n_estimators,
                               n_jobs=-1,
                               oob_score=True,
                               max_features=max_features, 
                               min_samples_leaf=min_samples_leaf)
    rf.fit(X, y)
    n = rfnnodes(rf)
    h = np.median(rfmaxdepths(rf))
    y_pred = rf.predict(X_valid)
    mae_valid = mean_absolute_error(np.exp(y_valid), np.exp(y_pred))
    rmsle_valid = np.sqrt( mean_squared_error(y_valid, y_pred) )
    r2_score_valid = rf.score(X_valid, y_valid)
    print(f"OOB R^2 {rf.oob_score_:.5f} using {n:,d} tree nodes {h} median tree height")
    print(f"Validation R^2 {r2_score_valid:.5f}, RMSLE {rmsle_valid:.5f}, MAE ${mae_valid:.0f}")
    return rf, r2_score_valid, rmsle_valid, mae_valid

In [ ]:
rf, r2_score_2007, rmsle_2007, mae_2007 =     test_valid(X, y, X_valid, y_valid)

In [ ]:
ntrees = 200
minleaf = 1
for maxf in np.arange(.1,.6,.1):
    print(f"n_estimators={ntrees}, max_features={maxf:.1f}, min_samples_leaf={minleaf}")
    test_valid(X, y, X_valid, y_valid,
               max_features=maxf, min_samples_leaf=minleaf)

In [ ]:
maxf = .3
for minleaf in range(2,7):
   print(f"n_estimators={ntrees}, max_features={maxf}, min_samples_leaf={minleaf}")
   test_valid(X, y, X_valid, y_valid,
              max_features=maxf, min_samples_leaf=minleaf)

In [ ]:
rf, r2_score_valid, rmsle_valid, mae_valid =     test_valid(X, y, X_valid, y_valid,
               max_features=.3, min_samples_leaf=2)

In [ ]:
I = importances(rf, X_valid, y_valid)
plot_importances(I.head(30))

In [ ]:
def select_features(X, y, X_valid, y_valid, drop=0.10):
   min_rmsle = 99999
   X_valid = X_valid.reindex(columns=X.columns)
   rf, _, rmsle, _ = test_valid(X, y, X_valid, y_valid,
                                max_features=.3, min_samples_leaf=2)
   I = importances(rf, X_valid, y_valid)
   features = list(I.index)
   keep = best_features = features
   n = int(.9/drop) # how many iterations? get to 90%
   for i in range(1,n+1):
       X2 = X[keep]
       X_valid2 = X_valid[keep]
       print(f"
Num features = {len(keep)}")
       rf2, _, rmsle, _ = test_valid(X2, y, X_valid2, y_valid,
                                     max_features=.3, min_samples_leaf=2)
       if rmsle < min_rmsle:
           min_rmsle = rmsle
           best_features = keep
       I2 = importances(rf2, X_valid2, y_valid) # recompute since collinear
       features = list(I2.index)
       keep = features[0:int(len(features)*(1-drop))]

   return min_rmsle, best_features

In [ ]:
best_features = ['age', 'ProductSize', 'fiProductClassSpec_lower', 'fiSecondaryDesc', 'YearMade', 'fiProductClassSpec_upper', 'Hydraulics_Flow_standard', 'fiModelDesc', 'fiBaseModel', 'ModelID', 'Enclosure_erops ac', 'fiProductClassSpec_units', 'fiModelDescriptor', 'age_na', 'ProductGroupDesc', 'YearMade_na', 'ProductGroup', 'Engine_Horsepower', 'Hydraulics', 'fiModelSeries', 'Enclosure_orops', 'MachineHoursCurrentMeter', 'fiProductClassDesc', 'Drive_System', 'state', 'auctioneerID', 'fiProductClassSpec_lower_na', 'Transmission', 'Track_Type', 'fiProductClassSpec_upper_na', 'Steering_Controls', 'Ripper', 'Ride_Control']

In [ ]:
X = X[best_features]
X_valid = X_valid[best_features]
rf, r2_score_bestf, rmsle_bestf, mae_bestf =     test_valid(X, y, X_valid, y_valid,
               max_features=.3, min_samples_leaf=2)

In [ ]:
y_valid_pred = rf.predict(X_valid)
underprediction = np.mean(y_valid-y_valid_pred)
dollars = np.mean(np.exp(y_valid)-np.exp(y_valid_pred))
print(f"Model underpredicts by ${dollars:.0f}, {underprediction:.5f} log(dollars)")

In [ ]:
!y_valid_pred = rf.predict(X_valid) + underprediction
mae_best = mean_absolute_error(np.exp(y_valid), np.exp(y_valid_pred))
rmsle_best = np.sqrt( mean_squared_error(y_valid, y_valid_pred) )
r2_score_best = r2_score(y_valid, y_valid_pred)
print(f"Adjusted-model validation R^2 {r2_score_best:.5f}, RMSLE {rmsle_best:.5f}, MAE {mae_best:.0f}")

In [ ]:
df = pd.read_feather("data/bulldozer-train-all.feather")
df = df.query('saledate.dt.year>=2007').copy()
X, y = df.drop('SalePrice', axis=1), df['SalePrice']
y = np.log(y)
clean(X)
X, catencoders = feature_eng(X)
medians = numericalize(X, catencoders)
X = X[best_features]

In [ ]:
df_test = pd.read_feather("data/bulldozer-test.feather")
X_test, y_test = df_test.drop('SalePrice', axis=1), df_test['SalePrice']
y_test = np.log(y_test)
clean(X_test)
X_test = feature_eng_test(X_test, catencoders)
df_apply_cats(X_test, catencoders)
df_fix_missing_test_nums(X_test, medians)
df_cat_to_catcode(X_test)
X_test = X_test[best_features]

In [ ]:
rf, r2_score_test, rmsle_test, mae_test =     test_valid(X, y + underprediction,
               X_test, y_test,
               max_features=.3, min_samples_leaf=2)

In [ ]:
scores = [
 ["Training 100k records", r2_valid_baseline, rmsle_valid_baseline, f"{mae_valid_baseline:.0f}"],
 ["Training set >= 2007", r2_score_2007, rmsle_2007, f"{mae_2007:.0f}"],
 ["After tuning", r2_score_valid, rmsle_valid, f"{mae_valid:.0f}"],
 ["Best feature subset", r2_score_bestf, rmsle_bestf, f"{mae_bestf:.0f}"],
 ["Inflation adjusted", r2_score_best, rmsle_best, f"{mae_best:.0f}"],
 ["Test set", r2_score_test, rmsle_test, f"{mae_test:.0f}"]
]
stats = pd.DataFrame(scores, columns=["Stage", "OOB R^2","RMSLE","MAE"])
stats = stats.set_index("Stage")